## **Team Reel Master Function -- Draft Working Version:**

- goes in parent directory
- runs when any new video or feedback is added (triggered by SQS)
- calls all other TeamReel functions (ML ones by us) + gets human feedback from DB
- Adds video analysis (all of the above) to the video_feedback table in the DB

In [ ]:
# Import modules/libraries we will use:

# Import libraries we will use:
from dotenv import load_dotenv
import json
import os
import psycopg2

# Import functions we need for TeamReel data infra, video and DB:
from data_infra.data_pipelines import get_next_video
from data_infra.postgresql_db_functions import get_feedback_for_user
from data_infra.postgresql_db_functions import get_feedback_for_video, get_video_info

# Import functions we need from audio_analysis package:
from audio_analysis.audio_functions import get_audio_from_video
from audio_analysis.audio_functions import analyse_audio, remove_files

# Import functions we need from facial_analysis package:
# To add

# Import functions we need from audio_analysis.background_noise module:
# To add


# ----------------------------------------------------------------------------
# SETUP:

# Get access info from .env file:
load_dotenv()

# PostgreSQL DB info:
PG_DB_HOST = os.getenv("PG_DB_HOST")
PG_DB_PORT = os.getenv("PG_DB_PORT")
PG_DB_NAME = os.getenv("PG_DB_NAME")
PG_DB_USER = os.getenv("PG_DB_USER")
PG_DB_PW = os.getenv("PG_DB_PW")
PG_DB_URI = os.getenv("PG_DB_URI")

# Open a connection to our PostgreSQL DB:
pg_conn = psycopg2.connect(
    host = PG_DB_HOST,
    port = PG_DB_PORT,
    database = PG_DB_NAME,
    user = PG_DB_USER,
    password = PG_DB_PW
)

# Instantiate a cursor using this connection:
pg_cursor = pg_conn.cursor()

# ----------------------------------------------------------------------------
# GET BASE MATERIALS: VIDEO, AUDIO, TRANSCRIPT:

# Get next video in line for analysis (recently uploaded by a user):
# (1) video_feedback dict = info about that video from our DB (video_id, etc.)
# (2) download .MP4 video file to project directory
video_feedback = get_next_video()

try:
    video_s3_key = video_info['video']['s3_key']
    video_filename = video_info['video']['s3_filename']
except KeyError:
    "KeyError: There is no information about this video in our database."

# Get audio from the video file:
get_audio_from_video(video_file=video_s3_filename)
audio_filename = 'audio.wav'

# ----------------------------------------------------------------------------
# SENTIMENT: AUDIO:

# Directory paths 
dirname = r"audio_chunks/"
path = r"text_chunks/"

# analyse_audio() function is the one function that does both: audio-sentiment analysis,
# and gets number of words per minute
data = analyse_audio() 

# Get the filename of the transcript
transcript_filename = 'outputfile.txt'
# Get the text in the transcript as a string
transcript_string = open(transcript_filename).read().replace("\n", " ")


# Add audio sentiment score to our video_feedback dict: 
video_feedback['sentiment_audio'] = data[0]['positive'] + data[0]['neutral'] # need to do some analysis and output 1 audio sentiment score
video_feedback['sentiment_audio_details'] = json.dumps(data[0])

# ----------------------------------------------------------------------------
# SPEED OF SPEECH: 

video_feedback['speaking_speed'] = round(data[2])


# ----------------------------------------------------------------------------
# SENTIMENT: VISUAL:

# [To add:] Facial centering: Call Chris Huskey's master function and get results
visual_sentiment_results = [To add]

# Add visual sentiment score to our video_feedback dict: 
video_feedback['sentiment_visual'] = ?? # need to do some analysis and output 1 visual sentiment score
video_feedback['sentiment_visual_details'] = visual_sentiment_results

# ----------------------------------------------------------------------------
# BACKGROUND NOISE:

# [To add:] Background noise: Call Chris Howell's master function and get results
# add to our video_feedback dict: video_feedback['']

# ----------------------------------------------------------------------------
# ADD VIDEO ANALYSIS RESULTS TO DB:

# [To add:] Add all of the above info for this video to the videos_feedback table in our DB

# pseudocode:
# if [video is in the video_feedback table already]:
#   Update info in the videos_feedback table
# else:
#   Add info to the video feedback table

# ----------------------------------------------------------------------------

# To add: Make Plotly visualizations so Web can display them super easily?

# ----------------------------------------------------------------------------
# REMOVE STORED FILES:

# To add: Remove local files (video, audio, transcript, audio chunks, (text chunks?))


In [ ]:
# {'video': {'video_id': 134,
#   'title': 'Kyla Oyamot',
#   's3_key': 'videos/ALPACAVID-i7swK-Wzc.webm',
#   's3_filename': 'ALPACAVID-i7swK-Wzc.webm',
#   'created_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
#   'updated_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))},
#  'prompt': {'prompt_id': 65,
#   'question': 'What is your first impression of Labs'},
#  'user': {'user_id': 185,
#   'first_name': 'Kyla',
#   'last_name': 'O',
#   'name': 'Kyla O',
#   'username': 'kylao'}}

In [ ]:
# id SERIAL PRIMARY KEY, 
# FOREIGN KEY (video_id) NOT NULL REFERENCES videos (id), 
# FOREIGN KEY (video_url) NOT NULL REFERENCES videos (video_url), 

# overall_performance NUMERIC, 

# sentiment_visual NUMERIC, 
# sentiment_visual_details JSONB, 
# sentiment_audio NUMERIC, 
# sentiment_audio_details JSONB, 
# sentiment_smile_frequency NUMERIC, 

# speaking_vocabulary NUMERIC, 
# speaking_speed NUMERIC, 

# background_noise NUMERIC, 

# appearance_facial_centering NUMERIC, 

# human_overall_performance NUMERIC, 
# human_delivery_and_presentation NUMERIC, 
# human_response_quality NUMERIC, 
# human_audio_quality NUMERIC, 
# human_visual_environment NUMERIC

In [ ]:
# - human score: overall rating
# - human score: delivery and presentation
# - human score: response quality
# - human score: audio quality
# - human score: visual environment
# - ML score: attitude
# - ML score: speaking speed
# - ML score: background noise
# - ML score: facial centering and posture